In [1]:
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [2]:
#Bring in the data

nba = pd.read_csv('./data/nba_analysis_data.csv')
nba.dropna(inplace = True)

In [3]:
#seperate test from train

nba_train = nba[nba.test == 0]
nba_test = nba[nba.test ==1]

In [4]:
nba_train = nba_train[nba_train.lastaway_cover == 1]
nba_test = nba_test[nba_test.lastaway_cover == 1]

In [5]:
X_train = nba_train.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv',
                       'home_starter5', 'teams', 'away_team',
                        'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_train = nba_train['home_win_margin']

In [6]:
X_test= nba_test.drop(columns = ['cover','home_win_margin', 
                        'date', 'dataset','line_cv', 
                       'home_starter5', 'teams', 'away_team',
                       'away_starter2', 'away_starter3',
                       'away_starter4', 'away_starter5',
                       'ref_1', 'ref_3', 'crew_referees',
                      'away_pace', 'away_spread',
                        'away_line_cv',  'away_cover',
                       'home_payout', 'away_payout'])

y_test = nba_test.home_win_margin

In [7]:
print(X_train.shape)
print(X_test.shape)

(1259, 141)
(248, 141)


In [8]:

#create interactions and squared terms
poly = PolynomialFeatures(degree = 2, interaction_only= True)
X_poly = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)
X_poly = pd.DataFrame(X_poly, 
                      columns = poly.get_feature_names(X_train.columns))
X_poly_test = pd.DataFrame(X_poly_test, 
                           columns = poly.get_feature_names(X_test.columns))


In [9]:
#scaling data to use in various other methods
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_poly)
X_test_scaled = ss.transform(X_poly_test)

In [10]:
from sklearn.decomposition import PCA


In [11]:
pc = PCA(n_components=200)
X_train_pc = pc.fit_transform(X_train_scaled)
X_test_pc = pc.transform(X_test_scaled)



In [12]:

var_exp = pc.explained_variance_ratio_
print('Explained variance:            ', var_exp)

cum_var_exp = np.cumsum(var_exp)
print('Cumulative explained variance: ', cum_var_exp)

Explained variance:             [0.09038483 0.08244363 0.04541493 0.03274802 0.03044167 0.0284749
 0.02685885 0.02351388 0.02302438 0.0219509  0.02064986 0.01951087
 0.01878104 0.01733658 0.01682348 0.0163952  0.01562775 0.01550776
 0.01496166 0.014814   0.01433441 0.01390898 0.01333591 0.01291501
 0.01272027 0.01211615 0.01202176 0.01158575 0.01143875 0.01127763
 0.0110337  0.01021517 0.01015566 0.01004279 0.00951187 0.00912319
 0.00906746 0.00889075 0.00825484 0.00794871 0.00765013 0.00728075
 0.00700972 0.00681743 0.0064582  0.00625342 0.00610071 0.00567107
 0.00546913 0.0054075  0.00498537 0.00429546 0.00410714 0.00391952
 0.00388373 0.00372464 0.00354189 0.00340442 0.00333304 0.00330413
 0.0028336  0.0026342  0.0024144  0.00230727 0.00212149 0.00210392
 0.00206032 0.00195775 0.00190512 0.00176257 0.00166949 0.00157
 0.00148125 0.00147254 0.00143085 0.00137902 0.00133791 0.00131616
 0.00125531 0.00115453 0.00111563 0.00107865 0.00104288 0.00100985
 0.00095944 0.00093491 0.00089974 

### Linear Regression

In [13]:
ols = LinearRegression()
ols.fit(X_train_pc, y_train)
ols.score(X_train_pc, y_train)

0.352165580560435

In [14]:
ols.score(X_test_pc, y_test)

0.01601355473464028

### Boosting

In [15]:
ad = AdaBoostRegressor()
ad_params = { 'n_estimators'      : [300],
             'learning_rate'         : [ .01 ],
             
            }

In [16]:
gs = GridSearchCV(ad, param_grid= ad_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'learning_rate': 0.01, 'n_estimators': 300}
0.10498121028051688


In [17]:
gs.score(X_test_pc,y_test)

0.09252887210791806

In [18]:
y_hat_ad = gs.predict(X_test_pc)

In [19]:
rr = RidgeCV(alphas = [70000, 90000, 80000])


In [20]:
rr.fit(X_train_pc, y_train);

In [21]:
rr.score(X_train_pc, y_train)

0.23737277440686932

In [22]:
rr.score(X_test_pc, y_test)

0.1604144945158631

In [23]:
y_hat_rr = rr.predict(X_test_pc)

In [24]:
rr.alpha_

70000

### Support Vector Machine

In [25]:
svm = SVR()
svm_params = {'kernel' : ['rbf'],
              'C'      : [.1, .02, .05]}

In [26]:
gs = GridSearchCV(svm, param_grid= svm_params)
gs.fit(X_train_pc, y_train)
print(gs.best_params_)
print(gs.best_score_)

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/bryancombs/anaconda3/lib/python3.6/site-packages/sklearn/svm/ba

{'C': 0.1, 'kernel': 'rbf'}
-0.008148520769042657


In [27]:
gs.score(X_test_pc, y_test)

-0.020379123144662392

In [28]:
y_hat_svm = gs.predict(X_test_pc)


### Neural Network

Going to fit a neural network with the X features.   I am not concerned about inference.  Accurate prediction is all that matters.  So am going to set up an neural network as one of the final predictors of Home Win Margin.

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [30]:
model = Sequential()
model.add(Dense(100, 
                activation = 'relu',
                input_dim = X_train_pc.shape[1]))
#model.add(Dense(18, activation = 'relu'))

model.add(Dense(1, activation = None)) #output layer 

In [31]:
model.compile(loss = 'mean_squared_error', optimizer='adam' )
model.fit(X_train_pc, 
          y_train, epochs =18, 
          batch_size= 64, 
          validation_data=(X_test_pc, y_test))

Train on 1259 samples, validate on 248 samples
Epoch 1/18
1259/1259 [==============================] - 0s 147us/step - loss: 219.2390 - val_loss: 197.3336
Epoch 2/18
1259/1259 [==============================] - 0s 25us/step - loss: 141.8973 - val_loss: 177.9379
Epoch 3/18
1259/1259 [==============================] - 0s 27us/step - loss: 121.7690 - val_loss: 175.5645
Epoch 4/18
1259/1259 [==============================] - 0s 29us/step - loss: 111.2209 - val_loss: 175.5595
Epoch 5/18
1259/1259 [==============================] - 0s 29us/step - loss: 102.8707 - val_loss: 173.7216
Epoch 6/18
1259/1259 [==============================] - 0s 25us/step - loss: 95.5892 - val_loss: 173.5327
Epoch 7/18
1259/1259 [==============================] - 0s 28us/step - loss: 89.3283 - val_loss: 174.8411
Epoch 8/18
1259/1259 [==============================] - 0s 25us/step - loss: 83.1403 - val_loss: 175.1213
Epoch 9/18
1259/1259 [==============================] - 0s 23us/step - loss: 77.9105 - val_loss: 17

In [32]:
y_hat_nn = model.predict(X_test_pc)
predictions = pd.DataFrame(y_hat_nn, columns= ['y_hat_nn'])
predictions

,y_hat_nn
0,3.941147
1,6.630055
2,7.141338
3,-5.787397
4,9.030302
5,-8.528008
6,-11.351385
7,-4.695289
8,-3.677318
9,-5.625159


In [33]:
predictions['y_hat_rr'] = y_hat_rr
predictions['y_hat_svm'] = y_hat_svm
predictions['y_hat_ad'] = y_hat_ad


In [34]:
predictions.head(10)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad
0,3.941147,10.414805,3.869231,5.607930
1,6.630055,-0.153959,3.869231,-2.854396
2,7.141338,4.983388,3.869231,-0.795276
3,-5.787397,1.637618,3.869231,-0.789011
4,9.030302,7.940345,3.869231,5.972509
5,-8.528008,-4.860900,3.869231,-2.042735
6,-11.351385,-3.064268,3.869231,-2.772300
7,-4.695289,-0.994245,3.869231,-1.629310
8,-3.677318,1.046049,3.869231,-0.516667
9,-5.625159,-0.969440,3.869231,-0.865248


In [35]:
y_s = pd.DataFrame(y_test)
y_s.reset_index(inplace = True)
predictions['y_test'] = y_s['home_win_margin']
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test
0,3.941147,10.414805,3.869231,5.607930,-14.0
1,6.630055,-0.153959,3.869231,-2.854396,-15.0
2,7.141338,4.983388,3.869231,-0.795276,-1.0
3,-5.787397,1.637618,3.869231,-0.789011,23.0
4,9.030302,7.940345,3.869231,5.972509,18.0


In [36]:
X_test.reset_index(inplace = True)
predictions['spread'] = X_test['spread']
predictions

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread
0,3.941147,10.414805,3.869231,5.607930,-14.0,-10.0
1,6.630055,-0.153959,3.869231,-2.854396,-15.0,3.5
2,7.141338,4.983388,3.869231,-0.795276,-1.0,-3.0
3,-5.787397,1.637618,3.869231,-0.789011,23.0,0.0
4,9.030302,7.940345,3.869231,5.972509,18.0,-7.5
5,-8.528008,-4.860900,3.869231,-2.042735,-1.0,8.5
6,-11.351385,-3.064268,3.869231,-2.772300,-10.0,4.5
7,-4.695289,-0.994245,3.869231,-1.629310,-3.0,2.0
8,-3.677318,1.046049,3.869231,-0.516667,-7.0,1.0
9,-5.625159,-0.969440,3.869231,-0.865248,-5.0,1.5


In [37]:
predictions['y_hat_ave'] = (predictions.y_hat_nn + 
                           
                            predictions.y_hat_rr + 
                            predictions.y_hat_nn)/3

In [38]:
predictions.head(20)

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave
0,3.941147,10.414805,3.869231,5.607930,-14.0,-10.0,6.099033
1,6.630055,-0.153959,3.869231,-2.854396,-15.0,3.5,4.368717
2,7.141338,4.983388,3.869231,-0.795276,-1.0,-3.0,6.422021
3,-5.787397,1.637618,3.869231,-0.789011,23.0,0.0,-3.312392
4,9.030302,7.940345,3.869231,5.972509,18.0,-7.5,8.666983
5,-8.528008,-4.860900,3.869231,-2.042735,-1.0,8.5,-7.305638
6,-11.351385,-3.064268,3.869231,-2.772300,-10.0,4.5,-8.589013
7,-4.695289,-0.994245,3.869231,-1.629310,-3.0,2.0,-3.461607
8,-3.677318,1.046049,3.869231,-0.516667,-7.0,1.0,-2.102862
9,-5.625159,-0.969440,3.869231,-0.865248,-5.0,1.5,-4.073253


In [39]:
predictions['predict_svm'] =  (predictions.y_hat_svm + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [40]:
predictions['predict_ad'] =  (predictions.y_hat_ad + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [41]:
predictions['predict_rr'] =  (predictions.y_hat_rr + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [42]:
predictions['predict_nn'] =  (predictions.y_hat_nn + predictions.spread).map(
    lambda x: 0 if x <0 else 1)

In [43]:
predictions['actual'] = (predictions.y_test + predictions.spread).map(lambda x: 0 if x< 0 else 1)

In [44]:
predictions['predict'] = (predictions['predict_svm']+ 
    predictions['predict_rr'] + predictions['predict_nn'])

In [45]:
predictions.head()

,y_hat_nn,y_hat_rr,y_hat_svm,y_hat_ad,y_test,spread,y_hat_ave,predict_svm,predict_ad,predict_rr,predict_nn,actual,predict
0,3.941147,10.414805,3.869231,5.607930,-14.0,-10.0,6.099033,0,0,1,0,0,1
1,6.630055,-0.153959,3.869231,-2.854396,-15.0,3.5,4.368717,1,1,1,1,0,3
2,7.141338,4.983388,3.869231,-0.795276,-1.0,-3.0,6.422021,1,0,1,1,0,3
3,-5.787397,1.637618,3.869231,-0.789011,23.0,0.0,-3.312392,1,0,1,0,1,2
4,9.030302,7.940345,3.869231,5.972509,18.0,-7.5,8.666983,0,0,1,1,1,2


In [46]:
print(classification_report(predictions.actual, predictions.predict_rr))

              precision    recall  f1-score   support

           0       0.51      0.38      0.44       129
           1       0.47      0.61      0.53       119

   micro avg       0.49      0.49      0.49       248
   macro avg       0.49      0.49      0.48       248
weighted avg       0.49      0.49      0.48       248



In [47]:
confusion_matrix(predictions.actual, predictions.predict_svm)

array([[59, 70],
       [55, 64]])

In [48]:
predictions.predict.value_counts(normalize = True)

2    0.358871
3    0.266129
1    0.209677
0    0.165323
Name: predict, dtype: float64

In [49]:
predictions.groupby(['predict'])['actual'].mean()

predict
0    0.512195
1    0.442308
2    0.483146
3    0.484848
Name: actual, dtype: float64